In [12]:
import pandas as pd
from jinja2 import Template
import os

# === CONFIGURATION ===
excel_file = "concepts.xlsx"           # Excel file with your concept data
template_folder = "templates"          # Folder containing your SQL templates
output_folder = "output_queries"       # Folder to store generated queries
combined_output = "All_Queries.sql"    # Combined output file name

# Make sure output folder exists
os.makedirs(output_folder, exist_ok=True)

# === STEP 1: Load Excel ===
df = pd.read_excel(excel_file)

# === STEP 2: Helper to pick correct template ===
def get_template_path(target_type):
    target_type = str(target_type).lower().strip()
    if target_type == "single":
        return os.path.join(template_folder, "base_script.sql")
    elif target_type == "target_reference":
        return os.path.join(template_folder, "target_reference_query.sql")
    else:
        return os.path.join(template_folder, "default_query.sql")

# === STEP 3: Generate queries and save individually ===
all_queries = []  # for combined output

for index, row in df.iterrows():
    target_type = str(row.get("TargetType", "")).strip().lower()
    concept_name = str(row.get("ConceptName", f"Concept_{index+1}")).strip()

    if not concept_name:
        print(f"⚠️ Skipping row {index+1}: missing ConceptName")
        continue

    # Pick template path
    template_path = get_template_path(target_type)

    if not os.path.exists(template_path):
        print(f"⚠️ Template not found for {target_type}, using default.")
        template_path = os.path.join(template_folder, "default_query.sql")

    # Read template
    with open(template_path, "r", encoding="utf-8") as f:
        template_text = f.read()

    template = Template(template_text)

    # Render query
    query = template.render(
        concept_name=concept_name,
        description=row.get("Description", ""),
        target_cpt_codes=row.get("TargetCPTCodes", ""),
        reference_cpt_codes=row.get("ReferenceCPTCodes", ""),
        claim_type=row.get("ClaimType", ""),
        dx_codes=row.get("DXCodes", "")
    )

    # Save per-concept query
    safe_name = concept_name.replace(" ", "_").replace("/", "_")
    output_path = os.path.join(output_folder, f"{safe_name}.sql")

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(query)

    print(f"✅ Created: {safe_name}.sql")

    # Add to combined output
    all_queries.append(f"-- ===== {concept_name} =====\n{query}\n\n")

# === STEP 4: Save combined queries ===
combined_path = os.path.join(output_folder, combined_output)
with open(combined_path, "w", encoding="utf-8") as f:
    f.writelines(all_queries)

print(f"\n🎉 All queries generated and combined successfully in: {combined_path}")


✅ Created: Concept_1.sql
✅ Created: Concept_2.sql

🎉 All queries generated and combined successfully in: output_queries\All_Queries.sql
